In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import os
from scipy.misc import imread,imresize
from random import shuffle
from sklearn.preprocessing import LabelEncoder
tf.__version__

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.5.0'

Make sure you download this data and extract in the same directory,

https://drive.google.com/open?id=1V9fy_Me9ZjmMTJoTWz0L8AUdIW5k35bE

In [2]:
checkpoint_name = 'mobilenet_v2_1.0_224'
url = 'https://storage.googleapis.com/mobilenet_v2/checkpoints/' + checkpoint_name + '.tgz'
print('Downloading from ', url)
!wget {url}
print('Unpacking')
!tar -xvf {checkpoint_name}.tgz
checkpoint = checkpoint_name + '.ckpt'

--2018-06-29 23:22:15--  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.31.112, 2404:6800:4001:80d::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.31.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78306834 (75M) [application/x-tar]
Saving to: ‘mobilenet_v2_1.0_224.tgz’

mobilenet_v2_1.0_22 100%[===================>]  74.68M  3.65MB/s    in 22s     

2018-06-29 23:22:37 (3.44 MB/s) - ‘mobilenet_v2_1.0_224.tgz’ saved [78306834/78306834]

Unpacking
./
./mobilenet_v2_1.0_224_eval.pbtxt
./mobilenet_v2_1.0_224.ckpt.data-00000-of-00001
./mobilenet_v2_1.0_224_frozen.pb
./mobilenet_v2_1.0_224.tflite
./mobilenet_v2_1.0_224_info.txt
./mobilenet_v2_1.0_224.ckpt.meta
./mobilenet_v2_1.0_224.ckpt.index


In [3]:
batch_size = 32
epoch = 10
learning_rate = 1e-3
data_location = 'Crop/'

In [4]:
img_lists = os.listdir(data_location)
shuffle(img_lists)
img_labels = [i.split('--')[0] for i in img_lists]
img_Y = LabelEncoder().fit_transform(img_labels)
img_lists = [data_location+i for i in img_lists]

In [5]:
import mobilenet_v2
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.placeholder(tf.float32,[None,224,224,1])
Y = tf.placeholder(tf.int32, [None])
images = tf.image.grayscale_to_rgb(X)
images = images / 128. - 1
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=True)):
    logits, endpoints = mobilenet_v2.mobilenet(images)
logits = tf.nn.relu6(logits)
emotion_logits = slim.fully_connected(logits, 7, activation_fn=None,
                                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                      weights_regularizer=slim.l2_regularizer(1e-5),
                                      scope='emo/emotion_1', reuse=False)
emotion_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=emotion_logits)
emotion_cross_entropy_mean = tf.reduce_mean(emotion_cross_entropy)
cost = tf.add_n([emotion_cross_entropy_mean] + tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
emotion_accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(emotion_logits, Y, 1), tf.float32))
global_step = tf.Variable(0, name="global_step", trainable=False)
# only train on our emotion layers
emotion_vars = [var for var in tf.trainable_variables() if var.name.find('emotion_') >= 0]
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost,var_list=emotion_vars)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'MobilenetV2')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, checkpoint)
saver = tf.train.Saver(tf.global_variables())
# test save
saver.save(sess, "new/emotion-checkpoint-mobilenet.ckpt")

INFO:tensorflow:Restoring parameters from mobilenet_v2_1.0_224.ckpt


'new/emotion-checkpoint-mobilenet.ckpt'

In [6]:
from tqdm import tqdm
batching = (len(img_lists) // batch_size) * batch_size
for i in range(epoch):
    total_loss, total_acc = 0, 0
    for k in tqdm(range(0, batching, batch_size),desc='minibatch loop'):
        batch_x = np.zeros((batch_size, 224,224,1))
        for n in range(batch_size):
            img = imresize(imread(img_lists[k+n]), (224,224))
            batch_x[n,:,:,0] = img
        loss, acc, _ = sess.run([cost,emotion_accuracy,optimizer],
                                feed_dict={X:batch_x,Y:img_Y[k:k+batch_size]})
        total_loss += loss
        total_acc += acc
    total_loss /= (len(img_lists) // batch_size)
    total_acc /= (len(img_lists) // batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1,total_loss,total_acc))
    saver.save(sess, "new/emotion-checkpoint-mobilenet.ckpt")

minibatch loop:   0%|          | 0/1104 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
minibatch loop: 100%|██████████| 1104/1104 [03:32<00:00,  5.19it/s]


epoch: 1, avg loss: 1.465783, avg accuracy: 0.507529


minibatch loop: 100%|██████████| 1104/1104 [03:32<00:00,  5.21it/s]


epoch: 2, avg loss: 1.353729, avg accuracy: 0.554235


minibatch loop: 100%|██████████| 1104/1104 [03:31<00:00,  5.21it/s]


...

epoch: 8, avg loss: 1.310964, avg accuracy: 0.569775


minibatch loop: 100%|██████████| 1104/1104 [03:31<00:00,  5.21it/s]


epoch: 9, avg loss: 1.310185, avg accuracy: 0.573171


minibatch loop: 100%|██████████| 1104/1104 [03:31<00:00,  5.21it/s]


epoch: 10, avg loss: 1.307986, avg accuracy: 0.573964


In [21]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.placeholder(tf.float32,[None,224,224,1])
Y = tf.placeholder(tf.int32, [None])
images = tf.image.grayscale_to_rgb(X)
images = images / 128. - 1
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
    logits, endpoints = mobilenet_v2.mobilenet(images)
logits = tf.nn.relu6(logits)
emotion_logits = slim.fully_connected(logits, 7, activation_fn=None,
                                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                      weights_regularizer=slim.l2_regularizer(1e-5),
                                      scope='emo/emotion_1', reuse=False)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())
saver.restore(sess, "new/emotion-checkpoint-mobilenet.ckpt")

INFO:tensorflow:Restoring parameters from new/emotion-checkpoint-mobilenet.ckpt


In [22]:
batching = (len(img_lists) // batch_size) * batch_size
results = []
for k in tqdm(range(0, batching, batch_size),desc='minibatch loop'):
    batch_x = np.zeros((batch_size, 224,224,1))
    for n in range(batch_size):
        img = imresize(imread(img_lists[k+n]), (224,224))
        batch_x[n,:,:,0] = img
    results += sess.run(tf.argmax(emotion_logits,1), feed_dict={X:batch_x}).tolist()

minibatch loop:   0%|          | 0/1104 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
minibatch loop: 100%|██████████| 1104/1104 [07:05<00:00,  2.60it/s]


In [23]:
from sklearn import metrics
print(metrics.classification_report(img_Y[:batching], results, target_names = np.unique(img_labels)))

             precision    recall  f1-score   support

         E1       0.31      0.21      0.25      6134
         E2       0.13      0.24      0.17      1401
         E3       0.23      0.32      0.27      5210
         E4       0.17      0.56      0.26      3273
         E5       0.41      0.26      0.32      7993
         E6       0.21      0.02      0.03      3286
         E7       0.38      0.25      0.30      8031

avg / total       0.31      0.26      0.26     35328

